<h1 align=center><font size = 5>Preparing the Data Frame</font></h1>

In [1]:
!conda install lxml --yes 
import requests
import lxml.html as lh
import pandas as pd

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    certifi-2019.11.28         |           py36_0         156 KB
    openssl-1.1.1d             |       h7b6447c_3         3.7 MB
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0     conda-forge --> 2019.11.28-py36_0   
    lxml:    4.2.5-py37hefd8a0e_0             --> 4.3.0-py36hefd8a0e_0
    openssl: 1.1.1d

In [2]:

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)


#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().rstrip()
    #print ('%d:"%s"'%(i,name))
    col.append((name,[]))

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    #print(len(T))
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data.rstrip())
        #Increment i for the next column
        i+=1

Dict={title:column for (title,column) in col}
df=pd.DataFrame.from_dict(Dict)
df.columns = ['PostalCode','Borough','Neighbourhood']
df.describe()


,PostalCode,Borough,Neighbourhood
count,287,287,287
unique,180,12,208
top,M8Y,Not assigned,Not assigned
freq,8,77,78


In [3]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df_without_Not_assigned=  df.loc[df['Borough'] != 'Not assigned']
df_without_Not_assigned.describe()


,PostalCode,Borough,Neighbourhood
count,210,210,210
unique,103,11,208
top,M9V,Etobicoke,Runnymede
freq,8,44,2


In [5]:
df_agg = df_without_Not_assigned.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
df_agg.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
import numpy as np
#df_agg.loc[df_agg['Neighbourhood'] == 'Not assigned'] == df_agg['Borough']
df_agg['Neighbourhood'] = np.where(df_agg['Neighbourhood']=='Not assigned', df_agg['Borough'] , df_agg['Neighbourhood'] )
df_agg.head(88)#['PostalCode'].where(df['PostalCode'] == 'M7A')

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Swansea, Runnymede"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre


In [7]:
df_agg.shape

(103, 3)

<h1 align=center><font size = 5>Getting Latitude and Longitude</font></h1>

In [8]:
df_lat_long = pd.read_csv("./Geospatial_Coordinates.csv") 
df_lat_long.columns = ['PostalCode','Latitude','Longitude']
df_lat_long.head()
df_merged = pd.merge(df_agg, df_lat_long, left_on='PostalCode', right_on='PostalCode')
df_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h1 align=center><font size = 5>Explore and cluster the neighborhoods in Toronto</font></h1>

In [9]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge


In [10]:
#df_merged_2=  df_merged.loc[df['Borough'] = 'Toronto']
df_merged_2 = df_merged[df['Borough'].str.contains("Toronto")]
df_merged_2.head()
df_merged_2


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
33,M3N,North York,Downsview Northwest,43.761631,-79.520999
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
67,M5T,Downtown Toronto,"Kensington Market, Grange Park, Chinatown",43.653206,-79.400049


In [15]:


latitude = 43.6532
longitude = -79.3832

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df_merged_2['Latitude'], df_merged_2['Longitude'], df_merged_2['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto) 
    
    
map_Toronto

The geograpical coordinate of Toronto are 43.6532, -79.3832.


In [16]:
CLIENT_ID = 'WZPXYQCODDYJWBUGIP02ICKMWTDDGBAGRLFCGTNQ0YPP5FPO' # your Foursquare ID
CLIENT_SECRET = 'GQTFU00JODTQMG1OSUQMYQYJT4AWBU42X2RWQDM3MSZ0PHJZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WZPXYQCODDYJWBUGIP02ICKMWTDDGBAGRLFCGTNQ0YPP5FPO
CLIENT_SECRET:GQTFU00JODTQMG1OSUQMYQYJT4AWBU42X2RWQDM3MSZ0PHJZ


In [90]:
#df_merged_2
df_merged_2 = df_merged_2.groupby('Neighbourhood').mean().reset_index()
df_merged_2.loc[4, 'Neighbourhood']

'Cliffside West, Birch Cliff'

In [91]:
neighborhood_latitude = df_merged_2.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_merged_2.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_merged_2.loc[4, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cliffside West, Birch Cliff are 43.692657000000004, -79.2648481.


In [92]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WZPXYQCODDYJWBUGIP02ICKMWTDDGBAGRLFCGTNQ0YPP5FPO&client_secret=GQTFU00JODTQMG1OSUQMYQYJT4AWBU42X2RWQDM3MSZ0PHJZ&v=20180605&ll=43.692657000000004,-79.2648481&radius=500&limit=100'

In [93]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df6f351618f43001b023e4f'},
 'response': {'headerLocation': 'Birch Cliff',
  'headerFullLocation': 'Birch Cliff, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.697157004500006,
    'lng': -79.25863612686784},
   'sw': {'lat': 43.6881569955, 'lng': -79.27106007313215}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56d51743498e60da346470e2',
       'name': 'The Birchcliff',
       'location': {'address': '1666 Kingston Road',
        'crossStreet': 'Birchcliff Avenue',
        'lat': 43.69166644406541,
        'lng': -79.26453158481682,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.69166644406541,
          'lng': -79.26453158481682}],
       

In [94]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [95]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframeprint('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [96]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Birchcliff,Café,43.691666,-79.264532
1,Birchmount Community Centre,General Entertainment,43.695175,-79.262161
2,Scarborough Gardens,Skating Rink,43.694647,-79.262230
3,Birchmount Stadium,College Stadium,43.695323,-79.261293


In [97]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [98]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
Toronto_venues = getNearbyVenues(names=df_merged_2['Neighbourhood'],
                                   latitudes=df_merged_2['Latitude'],
                                   longitudes=df_merged_2['Longitude']
                                  )


Berczy Park
Canada Post Gateway Processing Centre
Cedarbrae
Central Bay Street
Cliffside West, Birch Cliff
Davisville
Downsview Northwest
Emery, Humberlea
High Park, The Junction South
Hillcrest Village
Humber Summit
Kensington Market, Grange Park, Chinatown
Lawrence Manor, Lawrence Heights
Mount Olive, South Steeles, Thistletown, Silverstone, Beaumond Heights, Albion Gardens, Humbergate, Jamestown
North Toronto West
Northwest
Parkdale, Roncesvalles
Queen's Park
Richview Gardens, Kingsview Village, St. Phillips, Martin Grove Gardens
South Niagara, Bathurst Quay, King and Spadina, Railway Lands, CN Tower, Harbourfront West, Island airport
Stn A PO Boxes 25 The Esplanade
Swansea, Runnymede
Underground city, First Canadian Place
Upper Rouge
Westmount
Weston


In [100]:
print(Toronto_venues.shape)
Toronto_venues.head()

(689, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar,43.646676,-79.374822,Steakhouse
1,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
2,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
3,Berczy Park,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
4,Berczy Park,43.644771,-79.373306,Goose Island Brewhouse,43.647329,-79.373541,Beer Bar


In [101]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
Canada Post Gateway Processing Centre,10,10,10,10,10,10
Cedarbrae,8,8,8,8,8,8
Central Bay Street,84,84,84,84,84,84
"Cliffside West, Birch Cliff",4,4,4,4,4,4
Davisville,35,35,35,35,35,35
Downsview Northwest,4,4,4,4,4,4
"Emery, Humberlea",2,2,2,2,2,2
"High Park, The Junction South",25,25,25,25,25,25


In [102]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 184 uniques categories.


In [103]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Persian Restaurant,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
Toronto_onehot.shape

(689, 185)

In [104]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Persian Restaurant,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.00,0.000000,0.018182,0.018182,0.036364,0.000000,0.000000,0.0,0.018182,0.018182,0.000000,0.036364,0.000,0.000000,0.018182,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.018182,0.036364,0.00,0.000000,0.000000,0.036364,0.000000,0.000000,0.018182,0.036364,0.090909,0.000000,0.00,0.018182,0.000000,0.018182,0.0,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036364,0.000000,0.000000,0.000000,0.018182,0.00,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.018182,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.018182,0.018182,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.018182,0.018182,0.018182,0.00000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.018182,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.018182,0

In [105]:
Toronto_grouped.shape

(25, 185)

In [106]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1         Cheese Shop  0.04
2              Bakery  0.04
3      Farmers Market  0.04
4  Seafood Restaurant  0.04


----Canada Post Gateway Processing Centre----
                      venue  freq
0               Coffee Shop   0.2
1                     Hotel   0.2
2       American Restaurant   0.1
3             Burrito Place   0.1
4  Mediterranean Restaurant   0.1


----Cedarbrae----
                 venue  freq
0  Fried Chicken Joint  0.12
1     Hakka Restaurant  0.12
2                 Bank  0.12
3               Bakery  0.12
4   Athletics & Sports  0.12


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1  Italian Restaurant  0.05
2      Sandwich Place  0.05
3                Café  0.04
4      Ice Cream Shop  0.04


----Cliffside West, Birch Cliff----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3      

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [108]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Farmers Market,Cocktail Bar,Cheese Shop,Bakery,Seafood Restaurant,Steakhouse,Beer Bar,Japanese Restaurant
1,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Fried Chicken Joint,Sandwich Place,American Restaurant,Mediterranean Restaurant,Gym,Burrito Place,Yoga Studio,Deli / Bodega
2,Cedarbrae,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Hakka Restaurant,Bank,Athletics & Sports,Bakery,Deli / Bodega,Doner Restaurant
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Ice Cream Shop,Fried Chicken Joint,Bubble Tea Shop,Gym / Fitness Center,Salad Place
4,"Cliffside West, Birch Cliff",General Entertainment,College Stadium,Skating Rink,Café,Cuban Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant


In [124]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 4, 0, 0], dtype=int32)

In [111]:
#Toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Persian Restaurant,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.00,0.000000,0.018182,0.018182,0.036364,0.000000,0.000000,0.0,0.018182,0.018182,0.000000,0.036364,0.000,0.000000,0.018182,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.018182,0.036364,0.00,0.000000,0.000000,0.036364,0.000000,0.000000,0.018182,0.036364,0.090909,0.000000,0.00,0.018182,0.000000,0.018182,0.0,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036364,0.000000,0.000000,0.000000,0.018182,0.00,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.018182,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.018182,0.018182,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.018182,0.018182,0.018182,0.00000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.018182,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.018182,0

In [136]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = df_merged_2
Toronto_merged = df_merged_2.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
Toronto_merged.dropna(inplace=True)
Toronto_merged = Toronto_merged.astype({"Cluster Labels": int})
Toronto_merged # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Café,Farmers Market,Cocktail Bar,Cheese Shop,Bakery,Seafood Restaurant,Steakhouse,Beer Bar,Japanese Restaurant
1,Canada Post Gateway Processing Centre,43.636966,-79.615819,0,Coffee Shop,Hotel,Fried Chicken Joint,Sandwich Place,American Restaurant,Mediterranean Restaurant,Gym,Burrito Place,Yoga Studio,Deli / Bodega
2,Cedarbrae,43.773136,-79.239476,0,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Hakka Restaurant,Bank,Athletics & Sports,Bakery,Deli / Bodega,Doner Restaurant
3,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Ice Cream Shop,Fried Chicken Joint,Bubble Tea Shop,Gym / Fitness Center,Salad Place
4,"Cliffside West, Birch Cliff",43.692657,-79.264848,0,General Entertainment,College Stadium,Skating Rink,Café,Cuban Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
5,Davisville,43.704324,-79.388790,0,Dessert Shop,Pizza Place,Sandwich Place,Café,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Seafood Restaurant,American Restaurant
6,Downsview Northwest,43.761631,-79.520999,0,Grocery Store,Gym / Fitness Center,Liquor Store,Athletics & Sports,Yoga Studio,Cuban Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
7,"Emery, Humberlea",43.724766,-79.532242,4,Furniture / Home Store,Baseball Field,Yoga Studio,Deli / Bodega,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
8,"High Park, The Junction South",43.661608,-79.464763,0,Mexican Restaurant,Café,Bar,Thai Restaurant,Fried Chicken Joint,Grocery Store,Bookstore,Italian Restaurant,Park,Fast Food Restaurant
9,Hillcrest Village,43.803762,-79.363452,0,Dog Run,Golf Course,Fast Food Restaurant,Mediterranean Restaurant,Pool,Athletics & Sports,Yoga Studio,Cuban Restaurant,Discount Store,Diner


In [139]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [140]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.644771,Café,Farmers Market,Cocktail Bar,Cheese Shop,Bakery,Seafood Restaurant,Steakhouse,Beer Bar,Japanese Restaurant
1,43.636966,Hotel,Fried Chicken Joint,Sandwich Place,American Restaurant,Mediterranean Restaurant,Gym,Burrito Place,Yoga Studio,Deli / Bodega
2,43.773136,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Hakka Restaurant,Bank,Athletics & Sports,Bakery,Deli / Bodega,Doner Restaurant
3,43.657952,Sandwich Place,Italian Restaurant,Café,Burger Joint,Ice Cream Shop,Fried Chicken Joint,Bubble Tea Shop,Gym / Fitness Center,Salad Place
4,43.692657,College Stadium,Skating Rink,Café,Cuban Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
5,43.704324,Pizza Place,Sandwich Place,Café,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Seafood Restaurant,American Restaurant
6,43.761631,Gym / Fitness Center,Liquor Store,Athletics & Sports,Yoga Studio,Cuban Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
8,43.661608,Café,Bar,Thai Restaurant,Fried Chicken Joint,Grocery Store,Bookstore,Italian Restaurant,Park,Fast Food Restaurant
9,43.803762,Golf Course,Fast Food Restaurant,Mediterranean Restaurant,Pool,Athletics & Sports,Yoga Studio,Cuban Restaurant,Discount Store,Diner
11,43.653206,Bar,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Dumpling Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Grocery Store


In [141]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,43.706876,Convenience Store,Yoga Studio,Cuban Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [142]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,43.706748,Rental Car Location,Bar,Yoga Studio,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [143]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,43.756303,Home Service,Pizza Place,Yoga Studio,Cuban Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
24,43.696319,Middle Eastern Restaurant,Intersection,Coffee Shop,Sandwich Place,Chinese Restaurant,Comfort Food Restaurant,Dog Run,Discount Store,Diner


In [144]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,43.724766,Baseball Field,Yoga Studio,Deli / Bodega,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
